In [20]:
from officelib.xllib import *
from officelib import const as c
from hello.hello import *
import os
import math

def download(ip, name, fp):
    while True:
        try:
            app = open_hello(ip)
            app.login()
            r = app.getdatareport_bybatchname(name)
        except TrueError:
            pass
        else:
            break
    with open(fp, 'wb') as f:
        f.write(r)

def find_column(rng, header):
    return rng.Find(header, SearchOrder=c.xlByColumns)

def fudge_column(cells):
    """ For some reason, cells.Find() can't find 
    the requested header if the header occurs in the 
    first column, so insert a dummy here """
    cells.Columns(1).Insert()

def header_data(cells, h):
    col = find_column(cells, h)
    xs = col.Offset(2, 1)
    ys = col.Offset(2, 2)
    return col, xs, ys

def make_elapsed(cells, x_start):
    form = "=(%s-%s)*24*60" % (addr(x_start), x_start.Address)
    x_start.Offset(1, 2).EntireColumn.Insert()
    e_start = x_start.Offset(1,2)
    e_end = e_start.Offset(x_start.End(c.xlDown).Row-x_start.Row + 1, 1)
    target = cells.Range(e_end, e_start)
    
    e_start.Offset(0, 1).Value = "Elapsed(min)"  # header
    e_start.Value=form
    e_start.AutoFill(target)
    e_start.EntireColumn.NumberFormat = "0.00"
    
    return e_start

def addr(r):
    return r.GetAddress(False, False)

def chart_data_rng(cells, xs, ys, xe=None, ye=None):
    xe = xe or xs.End(c.xlDown)
    ye = ye or ys.Offset(xe.Row-xs.Row+1, 1)
    x = cells.Range(xs, xe)
    y = cells.Range(ys, ye)
    return x, y

def min_and_max(cells, x_start, ystart):
    rng = cells.Range(x_start, x_start.End(c.xlDown))
    yrng = cells.Range(ystart, ystart.End(c.xlDown))
    x = [x1[0] for x1 in rng.Value]
    y = [y1[0] for y1 in yrng.Value]
    xmin = min(x)
    xmax = max(x)
    ymin = min(y)
    ymax = max(y)
    return xmin, xmax, ymin, ymax

def shrink_markers(c, sz=2):
    for s in c.SeriesCollection():
        s.MarkerSize=sz

def analyze_rtd_data(pth, title, end=240):
    
    fp = os.path.abspath(pth)
    
    # Init ctx
    xl = Excel(visible=False)
    with HiddenXl(xl):
        wb = xl.Workbooks.Open(fp)
        ws = wb.Worksheets(1)
        cells = ws.Cells  
        return _analyze_rtd_data_inner(xl, wb, ws, cells, title, end)
    
def find_last_between(cells, ystart, yend, low, high):
    if addr(ystart) == addr(yend):
        # sanity check.
        return ystart
    data = cells.Range(ystart, yend).Value
    for i, (t,) in reversed(tuple(enumerate(data, 1))):
        if not (low <= t <= high):
            break
    return ystart.Offset(i, 1)

def find_last_smaller(cells, start, value):
    end = start.End(c.xlDown)
    if value is None:
        return end
    d = cells.Range(start, end).Value
    for i, (v,) in enumerate(d, 1):
        if v >= value:
            break
    return start.Offset(i, 1)

def ave(o):
    return sum(o)/len(o)


def find_stable_offset(cells, ta_y, ta_endy, tb_y, tb_endy):
    
    def data(s, e):
        return [x[0] for x in cells.Range(s, e).Value]
    
    ta_last = ta_endy.Value
    tb_last = tb_endy.Value
    margin = 0.05
    ta_stable_end = find_last_between(cells, ta_y, ta_endy, ta_last-margin, ta_last+margin)
    tb_stable_end = find_last_between(cells, tb_y, tb_endy, tb_last-margin, tb_last+margin)
    ta_d = data(ta_y, ta_y.Offset(3, 1))
    tb_d = data(tb_y, tb_y.Offset(3, 1))
    return ave(tuple((a-b) for a, b in zip(reversed(ta_d), reversed(tb_d))))

def make_offset_col(cells, ystart, os):
    form = "=%s+%s" % (addr(ystart), os)
    ystart.Offset(1, 2).EntireColumn.Insert()
    ostart = ystart.Offset(1, 2)
    oend = ystart.End(c.xlDown).Offset(1, 2)
    target = cells.Range(ostart, oend)
    ostart.Offset(0, 1).Value = "TempB offset corrected"
    ostart.Value = form
    ostart.AutoFill(target)
    return ostart

def make_diff_col(cells, ta_y, tb_y):
    tb_y.Offset(1, 2).EntireColumn.Insert()
    td_y=tb_y.Offset(1,2)
    td_h=td_y.Offset(0, 1)
    td_endy = tb_y.End(c.xlDown).Offset(1, 2)
    
    td_h.Value = "Lag (Water - RTD)"
    form = "=%s-%s" % (addr(ta_y), addr(tb_y))
    td_y.Value = form
    target = cells.Range(td_y, td_endy)
    td_y.AutoFill(target)
    return td_y, td_endy


def _analyze_rtd_data_inner(xl, wb, ws, cells, title, end):
    fudge_column(cells)
    c1 = create_charts(xl, wb, ws, cells, title, end)
    return c1


def fmt_x1(ch):
    xax = ch.Axes(c.xlCategory)
    xax.TickLabels.NumberFormat = "0"
    xax.HasMajorGridlines=True
    xax.MajorUnit = 15
    return xax
    
def mk_secondary(chart, series, txt):
    series.AxisGroup=c.xlSecondary
    y2ax = chart.Axes(c.xlValue, c.xlSecondary)
    y2ax.HasTitle=True
    y2ax.AxisTitle.Text = txt
    return y2ax

def nrows2(start, end):
    return end.Row - start.Row + 1

def nrows(cell, max_rows=None):
    n = nrows2(cell.End(c.xlDown), cell)
    if max_rows is not None:
        return min(n, max_rows)
    
def mk_cr(cells, cell, rows):
    return cells.Range(cell, cell.Offset(rows, 1))

class ChartData():
    def __init__(self, cells, header, max_rows=None):
        self.cells = cells
        self.header, self.xs, self.ys = header_data(cells, header)
        self.rows = nrows(self.xs, max_rows)
        self.es = None
        self.os = None
        self.osv = None
        
    def set_header(self, v):
        self.header.Value = v
        
    def make_offset(self, ref_start, ref_end):
        self.osv = find_stable_offset(self.cells, ref_start, ref_end, self.ys, self.ys.Offset(nrows2(ref_start, ref_end), 1))
        self.os = make_offset_col(self.cells, self.ys, self.osv)
        
    @property
    def x(self):
        return mk_cr(self.cells, self.xs, self.rows)
    
    @property
    def y(self):
        return mk_cr(self.cells, self.ys, self.rows)
    
    @property
    def e(self):
        return mk_cr(self.cells, self.es, self.rows)
    
    def plot_xy(self, chart):
        return CreateDataSeries(chart, self.x, self.y, self.header.Value)
    
    def plot_ey(self, chart):
        return CreateDataSeries(chart, self.e, self.y, self.header.Value)
    
    def make_elapsed(self):
        self.es = make_elapsed(self.cells, self.xs)
        
    def end_after_elapsed(self, m=240):
        assert self.es is not None, "Must have Elapsed column"
        ende = find_last_smaller(self.cells, self.es.End(c.xlDown), m)
        self.rows = nrows2(self.es, ende)
   
    
def create_charts(xl, wb, ws, cells, title, end):
    c1 = CreateChart(ws)
    
    ta, ta_x, ta_y = header_data(cells, "TempA(C)")
    tb, tb_x, tb_y = header_data(cells, "TempB(C)")
    hd, hd_x, hd_y = header_data(cells, "TempHeatDutyActual(%)")
    ta.Value = "Water Temp"
    tb.Value = "RTD Temp"
    
    ta_e = make_elapsed(cells, ta_x)
    tb_e = make_elapsed(cells, tb_x)
    hd_e = make_elapsed(cells, hd_x)
    
    ta_endx = find_last_smaller(cells, ta_e, end)
    ta_endy = ta_endx.Offset(1, 2)
    tb_endx = find_last_smaller(cells, tb_e, end)
    tb_endy = tb_endx.Offset(1, 2)
    
    rows = ta_endx.Row-ta_x.Row+1
    
    os = find_stable_offset(cells, ta_y, ta_endy, tb_y, tb_endy)
    o_y = make_offset_col(cells, tb_y, os)
    o_endy = o_y.Offset(rows, 1)

    hd_endy = hd_y.Offset(rows, 1)
    hd_ende = hd_endy.Offset(1, 0)
    
    x, y = chart_data_rng(cells, ta_e, ta_y, ta_endx, ta_endy)
    x2, y2 = chart_data_rng(cells, tb_e, o_y, tb_endx, o_endy)
    x4, y4 = chart_data_rng(cells, hd_e, hd_y, hd_ende, hd_endy)
    s1 = CreateDataSeries(c1, x, y, ta)
    s2 = CreateDataSeries(c1, x2, y2, tb)
    s4 = CreateDataSeries(c1, x4, y4, hd)
    
    fmt_x1(c1)
    mk_secondary(c1, s4, "Heat Duty (%)")
    c1.Axes(c.xlValue).MajorUnit = 1  # yaxis
    
    FormatChart(c1, None, title + " Over Time", "Elapsed(min)", "Temp(C)", False, True)
    shrink_markers(c1)
    xmin, xmax, ymin, ymax = min_and_max(cells, ta_e, ta_y)
    FormatAxesScale(c1, None, None, int(round(ymin-1, 1)), int(round(ymax+1, 1)))
        
    c1.Location(c.xlLocationAsNewSheet, "HeatRamp")
    c1 = wb.Charts("HeatRamp")
    
    c2 = CreateChart(ws)
    td_y, td_endy = make_diff_col(cells, ta_y, o_y)
    x3, y3 = chart_data_rng(cells, ta_e, td_y, ta_endx, td_endy)
    s3 = CreateDataSeries(c2, x3, y3, "Lag")
    s7 = CreateDataSeries(c2, x2, y2, "Temp(C)")
    
    fmt_x1(c2)
    y7ax2 = mk_secondary(c2, s7, "Temp(C)")
    
    FormatChart(c2, None, title + " Temp Lag", "Elapsed(min)", "Lag(C)", False, True)
    shrink_markers(c2)
    xmin, xmax, ymin, ymax = min_and_max(cells, ta_e, td_y)
    FormatAxesScale(c2, None, None, round(ymin, 1)-.5, round(ymax, 1)+.5)
    y7ax2.MinimumScale = 20
    y7ax2.MaximumScale = 40
    
    
    c2_name = "HeatLagvTime"
    c2.Location(c.xlLocationAsNewSheet, c2_name)
    c2 = wb.Charts(c2_name)
    
    c3 = CreateChart(ws)
    x5, y5 = chart_data_rng(cells, hd_y, td_y, hd_endy, td_endy)
    s5 = CreateDataSeries(c3, x5, y5)
    FormatChart(c3, None, title + " Heat vs. Lag", "Heat Duty (%)", "Lag(C)", False, False)
    shrink_markers(c3)
    
    c3_name = "HeatvLag"
    c3.Location(c.xlLocationAsNewSheet, c3_name)
    c3 = wb.Charts(c3_name)
    
    c4 = CreateChart(ws)
    x6, y6 = chart_data_rng(cells, ta_y, td_y, ta_endy, td_endy)
    s6 = CreateDataSeries(c4, x6, y6)
    FormatChart(c4, None, title + " Temp vs. Lag", "Temp(C)", "Lag(C)", False, False)
    shrink_markers(c4)
    xmin, xmax, ymin, ymax = min_and_max(cells, ta_y, td_y)
    FormatAxesScale(c4, int(xmin) - 2, int(xmax)+2, round(ymin, 1)-0.5, round(ymax, 1)+.5)
    
    c4_name = "TempvLag"
    c4.Location(c.xlLocationAsNewSheet, c4_name)
    c4 = wb.Charts(c4_name)   

In [13]:
def download_and_analyze(ip, name, fp, title, end=None):
    if not os.path.exists(fp):
        print("Downloading", name)
        download(ip, name, fp)
    print("Analyzing...")
    analyze_rtd_data(fp, title, end)
    print("Done")

In [14]:
from datetime import datetime as dt
os.chdir("C:/.replcache")

In [15]:
ip = "71.189.82.196:6"
name='rtdtest18c.1'
fp = name + " " + dt.now().strftime("%y%m%d%H%M") + ".csv"
title = "15L Heat Ramp Temp B Control"
end = 240

download_and_analyze(ip, name, fp, title, end)

Analyzing...
Done


In [16]:
fp = "rtdtest14f.1 1605271607.csv"
title = "15L Ambient RTD Effect, Single Heater #2, 8L"
analyze_rtd_data(name, title, 2)

com_error: (-2147352567, 'Exception occurred.', (0, 'Microsoft Office Excel', "'C:\\.replcache\\rtdtest18c.1' could not be found. Check the spelling of the file name, and verify that the file location is correct.\n\nIf you are trying to open the file from your list of most recently used files, make sure that the file has not been renamed, moved, or deleted.", 'C:\\Program Files\\Microsoft Office\\Office12\\1033\\XLMAIN11.CHM', 0, -2146827284), None)

In [ ]:
def animate_chart(step=5):
    xl = Excel()
    wb = xl.ActiveWorkbook
    ws = wb.Worksheets(1)
    cells = ws.Cells
    cr = cells.Range

    try:
        ch = wb.Charts("Test")
    except: 
        ch = CreateChart(ws)
        ch.Location(c.xlLocationAsNewSheet, "Test")
        ch = wb.Charts("Test")

    xtop = cr("K2")
    ytop = cr("L2")
    dtop = cr("H2")
    
    def get_date(c, r):
        return c.Offset(r, 1).Value.strftime("%I:%M%p %m/%d/%y").lower()

    def get_range(c, r):
        return cr(c.Offset(min(2, r), 1), c.Offset(r, 1))

    PurgeSeriesCollection(ch)
    end_row = xtop.End(c.xlDown).Row
    x = get_range(xtop, 1)
    y = get_range(ytop, 1)
    s=CreateDataSeries(ch, x, y)
    shrink_markers(ch)
    FormatChart(ch, Trendline=True)

    for i in range(step, end_row-1, step):
        x = get_range(xtop, i)
        y = get_range(ytop, i)
        s.XValues = x
        s.Values = y
        print("\r", i, "/", end_row, get_date(dtop, i), end="")
    i = end_row - 1
    x = get_range(xtop, i)
    y = get_range(ytop, i)
    s.XValues = x
    s.Values = y
    print("\r", i+1, "/", end_row, get_date(dtop, i), end="")

In [27]:
def create_charts2(xl, wb, ws, cells, title, end):
    c1 = CreateChart(ws)
    
    ta = ChartData(cells, "TempA(C)")
    tb = ChartData(cells, "TempB(C)")
    hd = ChartData(cells, "TempHeatDutyActual(%)")
    ta.set_header("Water Temp")
    tb.set_header("RTD Temp")
    
    for d in (ta, tb, hd):
        d.make_elapsed()
        d.end_after_elapsed(end)
    
    tb.make_offset(ta.y, ta.y.Offset(ta.rows, 1))

    #hd_endy = hd_y.Offset(rows, 1)
    #hd_ende = hd_endy.Offset(1, 0)
    
    #x, y = chart_data_rng(cells, ta_e, ta_y, ta_endx, ta_endy)
    #x2, y2 = chart_data_rng(cells, tb_e, o_y, tb_endx, o_endy)
    #x4, y4 = chart_data_rng(cells, hd_e, hd_y, hd_ende, hd_endy)
    s1 = ta.plot_ey(c1)
    s2 = tb.plot_ey(c1)
    s4 = hd.plot_ey(c1)
    fmt_x1(c1)
    mk_secondary(c1, s4, "Heat Duty (%)")
    c1.Axes(c.xlValue).MajorUnit = 1  # yaxis
    
    FormatChart(c1, None, title + " Over Time", "Elapsed(min)", "Temp(C)", False, True)
    shrink_markers(c1)
    xmin, xmax, ymin, ymax = min_and_max(cells, ta_e, ta_y)
    FormatAxesScale(c1, None, None, int(round(ymin-1, 1)), int(round(ymax+1, 1)))
        
    c1.Location(c.xlLocationAsNewSheet, "HeatRamp")
    c1 = wb.Charts("HeatRamp")
    return
    c2 = CreateChart(ws)
    td_y, td_endy = make_diff_col(cells, ta_y, o_y)
    x3, y3 = chart_data_rng(cells, ta_e, td_y, ta_endx, td_endy)
    s3 = CreateDataSeries(c2, x3, y3, "Lag")
    s7 = CreateDataSeries(c2, x2, y2, "Temp(C)")
    
    fmt_x1(c2)
    y7ax2 = mk_secondary(c2, s7, "Temp(C)")
    
    FormatChart(c2, None, title + " Temp Lag", "Elapsed(min)", "Lag(C)", False, True)
    shrink_markers(c2)
    xmin, xmax, ymin, ymax = min_and_max(cells, ta_e, td_y)
    FormatAxesScale(c2, None, None, round(ymin, 1)-.5, round(ymax, 1)+.5)
    y7ax2.MinimumScale = 20
    y7ax2.MaximumScale = 40
    
    
    c2_name = "HeatLagvTime"
    c2.Location(c.xlLocationAsNewSheet, c2_name)
    c2 = wb.Charts(c2_name)
    
    c3 = CreateChart(ws)
    x5, y5 = chart_data_rng(cells, hd_y, td_y, hd_endy, td_endy)
    s5 = CreateDataSeries(c3, x5, y5)
    FormatChart(c3, None, title + " Heat vs. Lag", "Heat Duty (%)", "Lag(C)", False, False)
    shrink_markers(c3)
    
    c3_name = "HeatvLag"
    c3.Location(c.xlLocationAsNewSheet, c3_name)
    c3 = wb.Charts(c3_name)
    
    c4 = CreateChart(ws)
    x6, y6 = chart_data_rng(cells, ta_y, td_y, ta_endy, td_endy)
    s6 = CreateDataSeries(c4, x6, y6)
    FormatChart(c4, None, title + " Temp vs. Lag", "Temp(C)", "Lag(C)", False, False)
    shrink_markers(c4)
    xmin, xmax, ymin, ymax = min_and_max(cells, ta_y, td_y)
    FormatAxesScale(c4, int(xmin) - 2, int(xmax)+2, round(ymin, 1)-0.5, round(ymax, 1)+.5)
    
    c4_name = "TempvLag"
    c4.Location(c.xlLocationAsNewSheet, c4_name)
    c4 = wb.Charts(c4_name)   

In [28]:
def _analyze_rtd_data_inner2(xl, wb, ws, cells, title, end):
    fudge_column(cells)
    c1 = create_charts2(xl, wb, ws, cells, title, end)
    return c1

def analyze_rtd_data2(pth, title, end=240):
    
    fp = os.path.abspath(pth)
    
    # Init ctx
    xl = Excel(visible=False)
    with HiddenXl(xl):
        wb = xl.Workbooks.Open(fp)
        ws = wb.Worksheets(1)
        cells = ws.Cells  
        return _analyze_rtd_data_inner2(xl, wb, ws, cells, title, end)

In [29]:
fp = os.path.abspath("rtdtest18c.1 1606091739.csv")
analyze_rtd_data2(fp, "test", 240)

NameError: name 'ta_e' is not defined